In [1]:
import argparse
from sys import exit, path
import os
import yaml
with open('dirs.yaml', 'r') as dir_settings_file:
    dir_settings = yaml.load(dir_settings_file, Loader=yaml.BaseLoader)
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import patsy as pt
import scipy.stats as st
import arviz as az
import theano

path.insert(1, "../../allen_src/")
import analysis_utils as utl
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from matplotlib.legend_handler import HandlerTuple
import matplotlib.ticker as ticker
from matplotlib.lines import Line2D
import matplotlib.colors
import seaborn as sns
import importlib
importlib.reload(utl);


measure = "tau_R"
analysis = 'allen_fc'
stimulus = 'movie'

defined_measures = ["tau_C",
                    "tau_R",
                    "R_tot"]
defined_stimuli = ["movie",
                   "spontaneous"]

seed = 12347

center = 'median' # measure of central tendency
T_measure = 'tau_R'
# T_measure = 'log_tau_R'
R_measure = 'R_tot'
# C_measure = 'mre_tau'
C_measure = 'tau_two_timescales'


if analysis == 'allen_bo':
    stimuli = ['natural_movie_three']
elif stimulus == "movie":
    stimuli = ['natural_movie_one_more_repeats']
elif stimulus == "spontaneous":
    stimuli = ['spontaneous']

T0 = 0.03 # 30 ms
#selected_structures = 'cortex+thalamus'
selected_structures = 'cortex'

plot_stars = False # plot stars or p-values for tests

stats_dir = dir_settings['stats_dir']

csv_file_name = "{}/{}_statistics_T0_30.csv".format(stats_dir, analysis)
# mre_stats_file_name = "{}/{}_mre_statistics_T0_30_Tmax_2500.csv".format(stats_dir, analysis)
mre_stats_file_name = "{}/{}_mre_statistics_T0_30_Tmax_10000_two_timescales.csv".format(stats_dir, analysis)

structures = ['VISp','VISl','VISrl','VISal','VISpm','VISam','LGd','LP']
# structures = ['VISp','VISl','VISrl','VISal','VISpm','VISam']
structures_map = utl.get_structures_map()
structures_cortex = [structure for structure in structures
                     if structures_map[structure]['parent_structure'] == 'cortex']

data_directory = dir_settings['allen_data_dir']
manifest_path = os.path.join(data_directory, "manifest.json")
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)


analysis_metrics = utl.get_analysis_metrics(cache,
                                            analysis)
data = utl.get_analysis_data(csv_file_name, analysis,
                             analysis_metrics=analysis_metrics,
                             mre_stats_file_name=mre_stats_file_name)

# make sure data as expected
if (selected_structures == 'cortex+thalamus') :
    _structures = structures
elif (selected_structures == 'cortex') :
    _structures = structures_cortex
else:
    print('unknown structures selected')
    exit()

data = data[utl.df_filter(data, structures=_structures, stimuli=stimuli, T0=T0, tmin=T0)]
try:
    num_neurons = utl.get_expected_number_of_neurons(analysis,
                                                     selected_structures,
                                                     stimuli)
    assert np.isclose(len(data), num_neurons, atol=100)
except:
    print(f'number of neurons is {len(data)}, expected {num_neurons}')
    exit()



if measure == "tau_C":
    measure = C_measure
    measure_name = r'intrinsic timescale'# $τ_{\mathregular{C}}$ (ms)'
    measure_name_short = '$τ_{\mathregular{C}}$ (ms)'
elif measure == "tau_R":
    measure = T_measure
    measure_name = 'information timescale'# $τ_R$ (ms)'
    measure_name_short = '$τ_R$'
elif measure == "R_tot":
    measure = R_measure
    measure_name = r'predictability'# $R_{\mathregular{tot}}$'
    measure_name_short = r'$R_{\mathregular{tot}}$'

selection = utl.get_data_filter(data, measure)
data = data[selection]


ModuleNotFoundError: No module named 'analysis_utils'